In [1]:
!pip install mysql-connector
!pip install requests
!pip install beautifulsoup4

     ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
      --------------------------------------- 0.3/11.9 MB ? eta -:--:--
      --------------------------------------- 0.3/11.9 MB ? eta -:--:--
      --------------------------------------- 0.3/11.9 MB ? eta -:--:--
      --------------------------------------- 0.3/11.9 MB ? eta -:--:--
     - ------------------------------------- 0.5/11.9 MB 275.1 kB/s eta 0:00:42
     - ------------------------------------- 0.5/11.9 MB 275.1 kB/s eta 0:00:42
     - ------------------------------------- 0.5/11.9 MB 275.1 kB/s eta 0:00:42
     -- ------------------------------------ 0.8/11.9 MB 325.8 kB/s eta 0:00:34
     -- ------------------------


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.7-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
Using cached soupsieve-2.7-py3-none-any.whl (36 kB)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from mysql import connector
import requests
from bs4 import BeautifulSoup

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

db_host = os.environ["DB_HOST"]
db_name = os.environ["DB_NAME"]
db_user = os.environ["DB_USER"]
db_password = os.environ["DB_PASSWORD"]

In [7]:
def connect_to_database(host=db_host, db=db_name, user=db_user, password=db_password):
    try:
        cnx = connector.connect(
            host=host,
            database=db,
            user=user,
            password=password
        )
        print("Connection to the database was successful.")
        return cnx
    except connector.Error as err:
        print(f"Error: {err}")
        return None

def execute_query(query):
    cnx = connect_to_database()
    if cnx is None:
        return None

    cursor = cnx.cursor()
    try:
        cursor.execute(query)
        cursor.commit()
        print("Query executed successfully.")
    except connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        cursor.close()
        cnx.close()

def db_return_results(query):
    cnx = connect_to_database()
    if cnx is None:
        return None

    cursor = cnx.cursor()
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        return results
    except connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        cursor.close()
        cnx.close()

In [8]:
import datetime
def date_checker():
    db_query = "select res_papers_date from res_papers order by res_papers_date desc limit 1"
    results = db_return_results(db_query)
    if datetime.year(datetime.datetime.now()) > datetime.year(results[0][0]) :
        print("The database is outdated.")
        return True
    else :
        print("The database is up to date.")
        return False

In [ ]:
def scrape_website():
    try:
        if date_checker():
            print("Fetching new data from the website...")
            links = db_return_results("select res_papers_links from res_papers where res_papers_links is not null;")
            if links:
                for link in links:
                    url = link[0]

                    soup = fetch_and_parse(url)
                    if soup:
                        # Process the soup object as needed
                        print(f"Successfully fetched and parsed {url}")

        db_execute_query("DELETE FROM res_papers")
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    except requests.RequestException as e:
        print(f"Error fetching the website: {e}")
        return None